In [2]:
import os
# 设置huggingface镜像源
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import re
import subprocess
from bs4 import BeautifulSoup
from openai import OpenAI
# 移除sentence_transformers导入
from sentence_transformers import SentenceTransformer
# 添加text2vec导入
# from text2vec import SentenceModel
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

d:\jupyter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# --- 1. 配置区域 ---

# 1.1 文档与数据库配置
SOURCE_DOC_FILE = "./yq2021-0112化解当下世界经济衰退的根本之道——基础设施建设.doc"
USE_MILVUS_LITE = False  # Docker 模式设为 False
MILVUS_HOST = "192.168.16.138"
MILVUS_PORT = "19530"
COLLECTION_NAME = "report_analysis_topic" # 新集合名称
# 修改为text2vec模型
# MODEL_NAME = 'text2vec-base-chinese'  # text2vec的中文模型
MODEL_NAME = 'shibing624/text2vec-base-chinese'  # 这个是在国内镜像上的中文模型

# 1.2 LLM API 配置 (请替换为您自己的服务商)
LLM_API_KEY = "sk-fa13f585000140deabdfa506b25a7f3d" # 替换这里
LLM_BASE_URL = "https://dashscope.aliyuncs.com/compatible-mode/v1"      # 替换这里
LLM_MODEL = "qwen-plus"                       # 替换这里

In [5]:
# --- 2. 文档加载模块 (复用之前的逻辑) ---
def find_libreoffice():
    possible_paths = [
        r"C:\Program Files\LibreOffice\program\soffice.exe",
        r"C:\Program Files (x86)\LibreOffice\program\soffice.exe",
        r"D:\LibreOffice\program\soffice.exe",
        "soffice"
    ]
    for path in possible_paths:
        try:
            if path != "soffice" and not os.path.exists(path): continue
            result = subprocess.run([path, '--version'], capture_output=True, text=True, timeout=10)
            if result.returncode == 0: return path
        except: continue
    return None

In [6]:
def load_doc_as_text(doc_path):
    print(f"正在读取原始文档: {doc_path}...")
    libreoffice_path = find_libreoffice()
    if not libreoffice_path: 
        print("❌ 未找到 LibreOffice")
        return None
    
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.dirname(doc_path)
    html_filename = os.path.basename(doc_path).rsplit('.', 1)[0] + '.html'
    html_path = os.path.join(output_dir, html_filename)
    
    if os.path.exists(html_path): os.remove(html_path)
    subprocess.run([libreoffice_path, '--headless', '--convert-to', 'html', '--outdir', output_dir, doc_path], capture_output=True)
    
    if not os.path.exists(html_path): return None
    
    content = ""
    for enc in ['utf-8', 'gb18030', 'gbk']:
        try:
            with open(html_path, 'r', encoding=enc) as f:
                content = f.read()
                break
        except: continue
            
    soup = BeautifulSoup(content, 'html.parser')
    text = soup.get_text()
    try: os.remove(html_path)
    except: pass
    return text

In [7]:
# --- 3. LLM 处理模块 (核心) ---
def generate_structured_summary(raw_text):
    """
    调用 LLM API，将原始文本重写为结构化格式
    """
    print("\n正在呼叫 LLM 进行文档重构与总结 (这可能需要几十秒)...")
    
    if "sk-xxxx" in LLM_API_KEY:
        print("❌ 错误：请先在脚本中设置有效的 LLM_API_KEY！")
        return None

    client = OpenAI(api_key=LLM_API_KEY, base_url=LLM_BASE_URL)

    # 定义系统提示词：强制 AI 输出特定格式
    system_prompt = """
    你是一个专业的文档分析师。请阅读用户提供的文档，并按照以下严格的格式重写和总结内容：
    1. 必须使用层级结构。一级标题使用中文数字（如“一、”、“二、”）。
    2. 一级标题下必须包含若干二级子主题，二级子主题以冒号结尾（如“明星IP化：”）。
    3. 必须保留原文中的关键数据、年份、观点和引用。
    4. 不要使用 Markdown 的加粗符号（**），直接输出纯文本。
    
    输出格式范例：
    一、发展背景
    时间节点：
    2014年发生了...
    二、核心乱象
    明星IP化：
    内容描述...
    饭圈化：
    内容描述...
    """

    try:
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"请处理以下文档内容：\n\n{raw_text[:8000]}"} # 防止超长，截取前8000字符
            ],
            temperature=0.3 # 较低温度，保证输出格式稳定
        )
        
        result = response.choices[0].message.content
        print("✅ LLM 重构完成！")
        return result
        
    except Exception as e:
        print(f"❌ LLM 调用失败: {e}")
        return None

In [8]:
# --- 4. 结构化切割逻辑 (复用) ---
def hierarchical_split(text):
    print("正在解析结构化文本...")
    chunks = []
    # 匹配 "一、" 开头的行作为一级分隔符
    level1_pattern = r"(?=(^[一二三四五六七八九十]、))"
    sections = re.split(level1_pattern, text.strip(), flags=re.MULTILINE)
    sections = [s.strip() for s in sections if s.strip()]

    for section in sections:
        lines = section.split('\n')
        if not lines: continue
        main_title = lines[0].strip()
        
        current_sub_title = "概述" # 默认子标题
        current_buffer = []

        for line in lines[1:]:
            line = line.strip()
            if not line: continue
            
            # 识别二级标题 (以冒号结尾)
            if len(line) < 30 and (line.endswith("：") or line.endswith(":")):
                if current_buffer:
                    # 保存上一块
                    full_context = f"【{main_title}】-> {current_sub_title}\n" + "\n".join(current_buffer)
                    chunks.append(full_context)
                
                current_sub_title = line.strip("：:")
                current_buffer = []
            else:
                current_buffer.append(line)
        
        # 保存最后一块
        if current_buffer:
            full_context = f"【{main_title}】-> {current_sub_title}\n" + "\n".join(current_buffer)
            chunks.append(full_context)
            
    return chunks

In [12]:
# --- 5. 主程序 ---
def main():
    # 1. 读取
    raw_text = load_doc_as_text(SOURCE_DOC_FILE)
    if not raw_text: return
    
    # 2. LLM 生成
    structured_text = generate_structured_summary(raw_text)
    if not structured_text: return
    
    # 打印一下 AI 生成的内容前几行，看看效果
    print("\n--- AI 生成内容预览 ---")
    print(structured_text[:500] + "...\n-----------------------\n")

    # 3. 切割
    chunks = hierarchical_split(structured_text)
    
    # 添加打印所有切割结果的代码
    print("\n=== 所有切割结果 ===")
    for i, chunk in enumerate(chunks, 1):
        print(f"\n--- 片段 {i} ---")
        print(chunk)
    print("\n=== 切割结果结束 ===")

    # 添加：切割完成后立即输出txt文档（带时间戳）
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"document_chunks_{timestamp}.txt"
    print(f"\n正在保存切割结果到文件: {output_filename}...")
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for i, chunk in enumerate(chunks, 1):
                f.write(f"\n--- 片段 {i} ---\n")
                f.write(f"{chunk}\n")
        print(f"✅ 切割结果已成功保存到: {output_filename}")
    except Exception as e:
        print(f"❌ 保存文件时出错: {e}")
    
    # 4. 存入 Milvus
    print(f"正在生成向量并存入 Milvus...")
    # 修改模型加载方式
    model = SentenceModel(MODEL_NAME)
    # text2vec的encode方法参数可能略有不同
    embeddings = model.encode(chunks)
    
    # 修改向量维度 - text2vec-base-chinese的维度是384
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384),  # 修改维度为384
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535)
    ]
    collection = Collection(name=COLLECTION_NAME, schema=CollectionSchema(fields))
    
    collection.insert([embeddings, chunks])
    collection.flush()
    collection.create_index("vector", {"metric_type": "L2", "index_type": "IVF_FLAT", "params": {"nlist": 128}})
    
    print(f"\n✅ 任务全流程完成！数据已存入集合 '{COLLECTION_NAME}'。")

if __name__ == "__main__":
    main()

正在读取原始文档: ./yq2021-0112化解当下世界经济衰退的根本之道——基础设施建设.doc...

正在呼叫 LLM 进行文档重构与总结 (这可能需要几十秒)...
✅ LLM 重构完成！

--- AI 生成内容预览 ---
一、发展背景  
时间节点：  
2020年新冠疫情席卷全球，全球经济按下“加速键”进入深度衰退。IMF预测2020年全球经济将收缩6%—7.6%，其中美国收缩8.0%，日本收缩5.8%，欧元区收缩10.2%，新兴经济体收缩2.5%。世界银行指出，本轮衰退为近百年来最严重的一次，全球近93%的经济体陷入衰退。叠加“逆全球化”趋势、中美关系恶化及全球经贸链条断裂，世界经济面临二战以来最为严峻的挑战。

核心判断：  
美联储主席鲍威尔称此次经济衰退“范围和速度尚无现代先例”，过去十年经济增长成果被抹去。疫情防控不力与发展中国家医疗基建短缺形成恶性循环，而发达国家亦难以快速响应基础设施建设需求。在此背景下，报告提出以“基础设施建设”作为破解全球经济困局的根本之道。

二、历史经验借鉴  
罗斯福新政挽救美国大萧条：  
1929—1933年大萧条期间，胡佛政府依赖市场自我调节失败，罗斯福上台后实施新政，启动25万个工程项目，包括修建7.8万座桥梁、105万公里道路、1127公里飞机跑道等。此举不仅安置了800万人就业，还将农村电力供应率从1935年的11%提升至1941年的40%以上。田纳...
-----------------------

正在解析结构化文本...

=== 所有切割结果 ===

--- 片段 1 ---
【一、发展背景】-> 时间节点
2020年新冠疫情席卷全球，全球经济按下“加速键”进入深度衰退。IMF预测2020年全球经济将收缩6%—7.6%，其中美国收缩8.0%，日本收缩5.8%，欧元区收缩10.2%，新兴经济体收缩2.5%。世界银行指出，本轮衰退为近百年来最严重的一次，全球近93%的经济体陷入衰退。叠加“逆全球化”趋势、中美关系恶化及全球经贸链条断裂，世界经济面临二战以来最为严峻的挑战。

--- 片段 2 ---
【一、发展背景】-> 核心判断
美联储主席鲍威尔称此次经济衰退“范围和速度尚无现代先例”，过去十年经济增长成果被抹去。疫情防控不力与发展中国家医疗基建短缺形成恶性循环，而发达国家亦难以快速响

NameError: name 'SentenceModel' is not defined